# Libraries

In [91]:
import pandas as pd
from getpass import getuser
from collections import defaultdict
from datetime import datetime, timedelta

# Load and inspect dataset

In [92]:
# Get the current user's name
user = getuser()

# Construct the path using the user's name
function_path = f"C:/Users/{user}/Documents/GitHub/tiebreak_wc/code/wiki/functions_uefa.ipynb"

# Run the notebook
%run $function_path


In [93]:


# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\in\eu_goals_men.xlsx'

# Load the Excel file into a DataFrame
df = pd.read_excel(data_path)

# Display the first few rows of the DataFrame to verify the import
display(df.head())

,stage,year,time,home_team,away_team,score,stadium_name,stadium_city,stadium_attendance,referee_name,...,goal_minute,extra_time,goals_home,goals_away,own_goal,penalty,goal_minute_et,goal_et,short_date,long_date
0,Semi-finals,1960,20:00,France,Yugoslavia,4–5,Parc des Princes,Paris,26370,Gaston Grandain,...,12,0,4,5,0,0,0,0,1960-07-06,6 July 1960
1,Semi-finals,1960,20:00,France,Yugoslavia,4–5,Parc des Princes,Paris,26370,Gaston Grandain,...,43,0,4,5,0,0,0,0,1960-07-06,6 July 1960
2,Semi-finals,1960,20:00,France,Yugoslavia,4–5,Parc des Princes,Paris,26370,Gaston Grandain,...,62,0,4,5,0,0,0,0,1960-07-06,6 July 1960
3,Semi-finals,1960,20:00,France,Yugoslavia,4–5,Parc des Princes,Paris,26370,Gaston Grandain,...,53,0,4,5,0,0,0,0,1960-07-06,6 July 1960
4,Semi-finals,1960,20:00,France,Yugoslavia,4–5,Parc des Princes,Paris,26370,Gaston Grandain,...,11,0,4,5,0,0,0,0,1960-07-06,6 July 1960


# Clean, transfrom, create variables

## time

In [94]:
# Step 1: Clean time variable by removing any letters (e.g., EEST, PST) if they exist
df['time_cleaned'] = df['time'].str.replace(r'[A-Za-z]+', '', regex=True).str.strip()

# Step 2: Split time where there are parentheses into local time and UTC offset
df['local_time'] = df['time_cleaned'].str.extract(r'(\d{2}:\d{2})')  # extract the part before parentheses
df['utc_time_offset'] = df['time_cleaned'].str.extract(r'\((.*?)\)')  # extract the part inside parentheses


# Extract relevant columns

In [95]:
# Filter out the knockout stages
df = df[~df['stage'].isin(['Quarter-finals', 'Round of 16', 'Semi-finals', 'Final'])]

# Extract relevant columns for goal events and match results
goals_df = df[['year', 'stage', 'home_team', 'away_team', 'scorer_nationality', 'goal_minute', 'short_date','local_time', 'score']]

# Remove duplicates
goals_df = goals_df.drop_duplicates()

display(goals_df.head())


,year,stage,home_team,away_team,scorer_nationality,goal_minute,short_date,local_time,score
12,1960,Third place play-off,Czechoslovakia,France,Czechoslovakia,58,1960-07-09,21:30,2–0
13,1960,Third place play-off,Czechoslovakia,France,Czechoslovakia,88,1960-07-09,21:30,2–0
23,1964,Third place play-off,Hungary,Denmark,Hungary,11,1964-06-20,20:30,3–1
24,1964,Third place play-off,Hungary,Denmark,Hungary,107,1964-06-20,20:30,3–1
25,1964,Third place play-off,Hungary,Denmark,Hungary,110,1964-06-20,20:30,3–1


# Recreate Leauge Table after first two matchdays

In [96]:

# Assuming goals_df is your original dataset
agg_goals_before_last_day, goals_last_day_sorted = process_goals_data(goals_df)


In [97]:
agg_goals_before_last_day

,year,stage,home_team,away_team,local_time,short_date,goals_home,goals_away,original_score,calculated_score,score_match,won
0,1980,Group 1,Czechoslovakia,West Germany,17:45,2024-11-11,0,1,0–1,0-1,True,-1
1,1980,Group 1,Greece,Czechoslovakia,20:30,2024-11-11,1,3,1–3,1-3,True,-1
2,1980,Group 1,Netherlands,Greece,20:30,2024-11-11,1,0,1–0,1-0,True,1
3,1980,Group 1,West Germany,Netherlands,17:45,2024-11-11,3,2,3–2,3-2,True,1
4,1980,Group 2,Belgium,England,17:45,2024-11-11,1,1,1–1,1-1,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...
161,2024,Group E,Slovakia,Ukraine,15:00,2024-11-11,1,2,1–2,1-2,True,-1
162,2024,Group F,Georgia,Czech Republic,15:00,2024-11-11,1,1,1–1,1-1,True,0
163,2024,Group F,Portugal,Czech Republic,21:00,2024-11-11,2,1,2–1,2-1,True,1
164,2024,Group F,Turkey,Georgia,18:00,2024-11-11,3,1,3–1,3-1,True,1


In [98]:
home_games, away_games = aggregate_home_away_points(agg_goals_before_last_day)

In [99]:
# Filter for year == 2021 and stage == 'Group F'
home2024d = home_games[(home_games['year'] == 2024) & (home_games['stage'] == 'Group D')]
home2024d

,year,stage,home_team,goals_scored,goals_conceded,points_home,match_count_home
132,2024,Group D,Austria,0,1,0,1
133,2024,Group D,Poland,2,5,0,2


In [100]:
# Filter for year == 2021 and stage == 'Group F'
away2024D = away_games[(away_games['year'] == 2024) & (away_games['stage'] == 'Group D')]
away2024D

,year,stage,away_team,goals_scored,goals_conceded,points_away,match_count_away
129,2024,Group D,Austria,3,1,3,1
130,2024,Group D,France,1,0,3,1
131,2024,Group D,Netherlands,2,1,3,1


## aggregate data after first two matches 

In [101]:
# Applying the fifa_before_last function to the mock data
all_games_before_last = uefa_before_last(home_games, away_games, agg_goals_before_last_day)

In [102]:
# Filtering all_games_before_last for the year 2021 and stage 'Group F'
group2024d = all_games_before_last[
    (all_games_before_last['year'] == 2024) & 
    (all_games_before_last['stage'] == 'Group D')
]

display(group2024d)

,year,stage,team,goals_scored,goals_conceded,points,goals_difference,total_matches,tiebreaker,tie_won,standing
171,2024,Group D,Netherlands,2,1,4,1,2,Netherlands,1,1
172,2024,Group D,France,1,0,4,1,2,Netherlands,0,2
173,2024,Group D,Austria,3,2,3,1,2,no need,0,3
174,2024,Group D,Poland,2,5,0,-3,2,no need,0,4


# Recreate league table after last match day

### uefa criteria 

In [103]:
# Initialize an empty list to store the results for each pair
all_results = []

# Get unique pairs of year, stage, and men
unique_pairs = all_games_before_last[['year', 'stage']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    year = row['year']
    stage = row['stage']
    
    # Apply the function to the current pair
    result = uefa_final_euro(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_results.append(result)

# Concatenate all the results into a single DataFrame
changes_df_euro = pd.concat(all_results)

# Keep only the specified columns
changes_df_euro = changes_df_euro[[ 'year', 'stage', 'team', '1st', '2nd', '3rd', '4th', 'changes']]



=== Initial Standings for Year 1980, Group 1 Before Last Match Goals ===

          team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  before_last_game_points  before_last_game_standing
  West Germany             5                   4                     2                      2                        4                          1
Czechoslovakia             3                   3                     2                      1                        2                          2
   Netherlands             3                   3                     3                      0                        2                          3
        Greece             1                   1                     4                     -3                        0                          4


Analyzing goal: 16 minute, Player team: Czechoslovakia, Home: Netherlands, Away: Czechoslovakia

=== Updated Standings After This Goal ===

          team  total_points  total_goals_scored  tot

In [104]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\wiki\men\uefa\eu\tb_eu_uefa_men.xlsx'
changes_df_euro.to_excel(file_path, index=False)


# group composition tracking

In [105]:
# Initialize an empty list to store the results for each pair
all_composition_changes = []

# Get unique pairs of year, stage, and men
unique_pairs = all_games_before_last[['year', 'stage']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    year = row['year']
    stage = row['stage']
    
    # Apply the function to the current pair
    composition_changes_df = track_composition_changes(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_composition_changes.append(composition_changes_df)

# Concatenate all the results into a single DataFrame
final_composition_changes_df = pd.concat(all_composition_changes, ignore_index=True)

# Rename 'player_team' column to 'scorer_team'
final_composition_changes_df.rename(columns={'player_team': 'scorer_nationality'}, inplace=True)

# Dropping the "scorer_team" column
final_composition_changes_df.drop(columns=['scorer_team'], inplace=True)




=== Initial Standings for Group 1, 1980 (with 0-0 points added) ===
          team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  before_last_game_standing
  West Germany             5                   4                     2                      2                          1
Czechoslovakia             3                   3                     2                      1                          2
   Netherlands             3                   3                     3                      0                          3
        Greece             1                   1                     4                     -3                          4



=== Standings after goal at minute 16 in Group 1, edition 1980 ===
          team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  last_game_standing  tied_won
  West Germany             5                   4                     2                      2                   1         1
Czechosl

In [106]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\wiki\men\uefa\eu\standings_eu_uefa_men.xlsx'
final_composition_changes_df.to_excel(file_path, index=False)


# best four third placed

In [107]:
# Call the function to get the DataFrame
final_df = best_four_third_placed_eu_men(goals_last_day_sorted, all_games_before_last, agg_goals_before_last_day)

# Define the file path and save to Excel
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\wiki\men\uefa\eu\third_teams_eu_uefa_men.xlsx'
final_df.to_excel(file_path, index=False)



--- Processing Year: 2016 ---

=== Initial Standings for Year 2016 Before Processing Any Goals ===

               team   stage  before_last_game_points  before_last_game_goals_scored  before_last_game_goals_conceded  total_goal_difference
             France Group A                        6                              4                                1                      3
        Switzerland Group A                        4                              2                                1                      1
            Romania Group A                        1                              2                                3                     -1
            Albania Group A                        0                              0                                3                     -3
            England Group B                        4                              3                                2                      1
              Wales Group B                        3       

In [108]:
# Count the number of observations where the year is 2016
count_2016 = goals_last_day_sorted[goals_last_day_sorted['year'] == 2016].shape[0]
print(f"Number of observations in goals_last_day_sorted for the year 2016: {count_2016}")

# Count the number of observations where the year is 2021
count_2020 = goals_last_day_sorted[goals_last_day_sorted['year'] == 2021].shape[0]
print(f"Number of observations in goals_last_day_sorted for the year 2020: {count_2020}")

# Count the number of observations where the year is 2024
count_2024 = goals_last_day_sorted[goals_last_day_sorted['year'] == 2024].shape[0]
print(f"Number of observations in goals_last_day_sorted for the year 2024: {count_2024}")


Number of observations in goals_last_day_sorted for the year 2016: 22
Number of observations in goals_last_day_sorted for the year 2020: 38
Number of observations in goals_last_day_sorted for the year 2024: 24
